This is based on ```Census_TonsorHerath_Dec20_2023.ipynb```.

 - We decided to add some more to it; irrigated hay acrage.
 - Filtered counties based on some criteria; $\text{RA} \ge 50K$ or [$\text{RA} \le 50K$ and $\text{RAF} \ge 10\%$]

### Variables we need:
 - ```Inventory```
 - ```NPP```
 - ```RA?```
 - ```herb?```
 - ```feed cost``` (in some form)
 - ```resident population```
 - ```irrigated/dryland area``` or just rangeland area (as percentage)
 
**Herath** 
 - ```slaughter numbers```
 - ```weather variables```
 - ```unemployment rate```
 - ```farmland availibility```
 - ```labor cost```
 - ```energy price```
 - ```value of farmland```
 
**Kirti**
 - ```Time of max NPP```
 - ```variation of NPP```

In [1]:
import shutup

shutup.please()

import pandas as pd
import numpy as np
from datetime import datetime
import os, os.path, pickle, sys
import seaborn as sns

import matplotlib
import matplotlib.pyplot as plt

from sklearn import preprocessing
import statistics
import statsmodels.api as sm

sys.path.append("/Users/hn/Documents/00_GitHub/Rangeland/Python_Codes/")
import rangeland_core as rc

In [2]:
data_dir_base = "/Users/hn/Documents/01_research_data/RangeLand/Data/"
census_population_dir = data_dir_base + "census/"
# Shannon_data_dir = data_dir_base + "Shannon_Data/"
# USDA_data_dir = data_dir_base + "/NASS_downloads/"
param_dir = data_dir_base + "parameters/"
Min_data_base = data_dir_base + "Min_Data/"
reOrganized_dir = data_dir_base + "reOrganized/"

In [3]:
# for bold print
start_b = "\033[1m"
end_b = "\033[0;0m"
print ("This is " + start_b + "a_bold_text" + end_b + "!")

This is a_bold_text!


# Read the data

In [4]:
SoI = [
    "Alabama",
    "Arizona",
    "Arkansas",
    "California",
    "Colorado",
    "Florida",
    "Georgia",
    "Idaho",
    "Illinois",
    "Iowa",
    "Kansas",
    "Kentucky",
    "Louisiana",
    "Mississippi",
    "Missouri",
    "Montana",
    "Nebraska",
    "Nevada",
    "New Mexico",
    "North Dakota",
    "Oklahoma",
    "Oregon",
    "South Dakota",
    "Tennessee",
    "Texas",
    "Utah",
    "Virginia",
    "Washington",
    "Wyoming",
]

abb_dict = pd.read_pickle(param_dir + "state_abbreviations.sav")
SoI_abb = []
for x in SoI:
    SoI_abb = SoI_abb + [abb_dict["full_2_abb"][x]]

#### County FIPS

In [5]:
county_fips = pd.read_pickle(reOrganized_dir + "county_fips.sav")

county_fips = county_fips["county_fips"]

print (f"{len(county_fips.state.unique()) = }")
county_fips = county_fips[county_fips.state.isin(SoI_abb)].copy()
county_fips.drop_duplicates(inplace=True)
county_fips.reset_index(drop=True, inplace=True)
print (f"{len(county_fips.state.unique()) = }")

county_fips.head(2)

len(county_fips.state.unique()) = 53
len(county_fips.state.unique()) = 29


,county_fips,county_name,fips,state,state_fip,EW
0,01001,Autauga County,1001,AL,01,E
1,01003,Baldwin County,1003,AL,01,E


In [6]:
cnty_interest_list = list(county_fips.county_fips.unique())

### Inventory

In [7]:
USDA_data = pd.read_pickle(reOrganized_dir + "USDA_data.sav")
inventory = USDA_data["cattle_inventory"]

# pick only the counties we want
# cattle_inventory = cattle_inventory[cattle_inventory.county_fips.isin(cnty_interest_list)].copy()

print(f"{inventory.data_item.unique() = }")
print(f"{inventory.commodity.unique() = }")
print()
print(f"{len(inventory.state.unique())= }")

inventory.head(2)

inventory.data_item.unique() = array(['CATTLE, COWS, BEEF - INVENTORY'], dtype=object)
inventory.commodity.unique() = array(['CATTLE'], dtype=object)

len(inventory.state.unique())= 29


,year,state,state_ansi,ag_district,ag_district_code,county,county_ansi,commodity,data_item,cattle_cow_beef_inventory,cattle_cow_beef_inventory_cv_(%),county_fips
0,2017,Alabama,01,BLACK BELT,40,Autauga,001,CATTLE,"CATTLE, COWS, BEEF - INVENTORY",8678.0,20.9,01001
1,2017,Alabama,01,BLACK BELT,40,Dallas,047,CATTLE,"CATTLE, COWS, BEEF - INVENTORY",14589.0,20.9,01047


In [8]:
inventory.rename(columns={"cattle_cow_beef_inventory": "inventory"}, inplace=True)

In [9]:
census_years = sorted(list(inventory.year.unique()))
print(f"{census_years = }")

# pick only useful columns
inv_col_ = "inventory"
inventory = inventory[["year", "county_fips", inv_col_]]

print(f"{len(inventory.county_fips.unique()) = }")
inventory.head(2)

census_years = [1997, 2002, 2007, 2012, 2017]
len(inventory.county_fips.unique()) = 2150


,year,county_fips,inventory
0,2017,01001,8678.0
1,2017,01047,14589.0


### See how many counties and how many data points are incomplete in inventory

In [10]:
all_cattle_counties = list(inventory.county_fips.unique())
# print(f"{len(all_cattle_counties) = }")
incomplete_counties = {}
for a_cnty_fip in all_cattle_counties:
    curr_cow = inventory[inventory.county_fips == a_cnty_fip].copy()
    missing_yr = [x for x in census_years if not(x in list(curr_cow.year))]
    if (len(missing_yr)>0):
        incomplete_counties[a_cnty_fip] = missing_yr
        
lic = len(incomplete_counties)
la = len(all_cattle_counties)
print ("There are {} incomlete counties out of {} for census years!!!".format(lic, la))

There are 1307 incomlete counties out of 2150 for census years!!!


In [11]:
{key:value for key,value in list(incomplete_counties.items())[0:3]}

{'01001': [2002], '01047': [2002, 2007], '01091': [2007, 2012]}

## NPP exist only after 2001! 
So let us use subset of cattle inventory from census

In [12]:
inventory = inventory[inventory.year>=2001]
inventory.reset_index(drop=True, inplace=True)

census_years = sorted(list(inventory.year.unique()))
inventory.head(2)

,year,county_fips,inventory
0,2017,01001,8678.0
1,2017,01047,14589.0


In [13]:
all_cattle_counties = list(inventory.county_fips.unique())
# print(f"{len(all_cattle_counties) = }")
incomplete_counties = {}
for a_cnty_fip in all_cattle_counties:
    curr_cow = inventory[inventory.county_fips == a_cnty_fip].copy()
    missing_yr = [x for x in census_years if not(x in list(curr_cow.year))]
    if (len(missing_yr)>0):
        incomplete_counties[a_cnty_fip] = missing_yr
        
lic = len(incomplete_counties)
la = len(all_cattle_counties)
print ("There are {} incomlete counties out of {} for census years!!!".format(lic, la))

There are 1143 incomlete counties out of 2095 for census years!!!


## Since there are too many incomlete counties, lets just keep them!

#### Rangeland area and Herb Ratio

In [14]:
RA = pd.read_csv(reOrganized_dir + "county_rangeland_and_totalarea_fraction.csv")
RA.rename(columns={"fips_id": "county_fips"}, inplace=True)
RA = rc.correct_Mins_county_FIPS(df=RA, col_ = "county_fips")
print (f"{len(RA.county_fips.unique()) = }")
RA = RA[RA.county_fips.isin(cnty_interest_list)]
print (f"{len(RA.county_fips.unique()) = }")
RA.reset_index(drop=True, inplace=True)
RA.head(2)

len(RA.county_fips.unique()) = 2379
len(RA.county_fips.unique()) = 1747


,county_fips,rangeland_acre,county_area_acre,rangeland_fraction
0,01003,13037.43,1060302.72,0.010000
1,01005,18.23,575781.12,0.000032


In [15]:
cnty_interest_list[:3]

['01001', '01003', '01005']

In [16]:
herb = pd.read_pickle(data_dir_base + "Supriya/Nov30_HerbRatio/county_herb_ratio.sav")
herb = herb["county_herb_ratio"]
herb.head(2)
print (herb.shape)
herb = herb[herb.county_fips.isin(cnty_interest_list)]
print (herb.shape)

herb.dropna(how="any", inplace=True)
print (herb.shape)

herb.reset_index(drop=True, inplace=True)
herb.head(3)

(3233, 4)
(2239, 4)
(1758, 4)


,county_fips,herb_avg,herb_std,pixel_count
0,01003,27.736637,25.907239,2563.0
1,01007,10.036212,16.976569,2154.0
2,01015,13.333333,13.960261,3.0


In [17]:
RA_herb = pd.merge(RA, herb, on=["county_fips"], how="left")
# RA_herb.dropna(how="any", inplace=True)
RA_herb.reset_index(drop=True, inplace=True)
RA_herb.head(2)

,county_fips,rangeland_acre,county_area_acre,rangeland_fraction,herb_avg,herb_std,pixel_count
0,01003,13037.43,1060302.72,0.010000,27.736637,25.907239,2563.0
1,01005,18.23,575781.12,0.000032,NaN,NaN,NaN


In [18]:
inventory_RA_herb = pd.merge(inventory, RA_herb, on=["county_fips"], how="left")
inventory_RA_herb.head(2)

,year,county_fips,inventory,rangeland_acre,county_area_acre,rangeland_fraction,herb_avg,herb_std,pixel_count
0,2017,01001,8678.0,NaN,NaN,NaN,NaN,NaN,NaN
1,2017,01047,14589.0,NaN,NaN,NaN,NaN,NaN,NaN


### NPP

In [19]:
cty_yr_GPP_NPP_prod = pd.read_csv(reOrganized_dir + "county_annual_GPP_NPP_productivity.csv")

cty_yr_GPP_NPP_prod.rename(columns={"county" : "county_fips",
                                    "MODIS_NPP" : "unit_npp"}, inplace=True)
cty_yr_GPP_NPP_prod = rc.correct_Mins_county_FIPS(df=cty_yr_GPP_NPP_prod, col_ = "county_fips")

print (f"{len(cty_yr_GPP_NPP_prod.county_fips.unique()) = }")
cty_yr_GPP_NPP_prod = cty_yr_GPP_NPP_prod[cty_yr_GPP_NPP_prod.county_fips.isin(cnty_interest_list)]
print (f"{len(cty_yr_GPP_NPP_prod.county_fips.unique()) = }")


cty_yr_GPP_NPP_prod.head(5)

len(cty_yr_GPP_NPP_prod.county_fips.unique()) = 2265
len(cty_yr_GPP_NPP_prod.county_fips.unique()) = 1688


,year,county_fips,MODIS_GPP,unit_npp,productivity
0,2001,01003,2.272050,0.926441,4163.385965
1,2001,01005,1.227967,0.629133,NaN
2,2001,01007,1.445234,0.641586,4620.363636
3,2001,01015,1.429812,0.736375,NaN
4,2001,01019,1.294283,0.631583,NaN


In [20]:
cty_yr_GPP_NPP_prod = pd.merge(cty_yr_GPP_NPP_prod, 
                               RA[["county_fips", "rangeland_acre"]], 
                               on=["county_fips"], how="left")

cty_yr_GPP_NPP_prod = rc.covert_unitNPP_2_total(NPP_df=cty_yr_GPP_NPP_prod, 
                                                npp_unit_col_ = "unit_npp", 
                                                acr_area_col_ = "rangeland_acre", 
                                                npp_area_col_ = "county_total_npp")

cty_yr_GPP_NPP_prod.head(2)

,year,county_fips,MODIS_GPP,unit_npp,productivity,rangeland_acre,area_m2,county_total_npp
0,2001,01003,2.272050,0.926441,4163.385965,13037.43,5.276069e+07,4.887964e+07
1,2001,01005,1.227967,0.629133,NaN,18.23,7.377431e+04,4.641388e+04


In [21]:
cty_yr_npp = cty_yr_GPP_NPP_prod[["year", "county_fips", "county_total_npp"]]
cty_yr_npp.dropna(how="any", inplace=True)
cty_yr_npp.reset_index(drop=True, inplace=True)
cty_yr_npp.head(2)

,year,county_fips,county_total_npp
0,2001,01003,4.887964e+07
1,2001,01005,4.641388e+04


In [22]:
inventory_RA_herb.head(2)

,year,county_fips,inventory,rangeland_acre,county_area_acre,rangeland_fraction,herb_avg,herb_std,pixel_count
0,2017,01001,8678.0,NaN,NaN,NaN,NaN,NaN,NaN
1,2017,01047,14589.0,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
inventory_RA_herb_NPP = pd.merge(inventory_RA_herb, cty_yr_npp, 
                                 on=["county_fips", "year"], how="left")

inventory_RA_herb_NPP.head(2)

,year,county_fips,inventory,rangeland_acre,county_area_acre,rangeland_fraction,herb_avg,herb_std,pixel_count,county_total_npp
0,2017,01001,8678.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2017,01047,14589.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## New Variables compared to old models

In [24]:
slaughter_Q1 = pd.read_pickle(reOrganized_dir + "slaughter_Q1.sav")
slaughter_Q1 = slaughter_Q1["slaughter_Q1"]
slaughter_Q1.rename(columns={"cattle_on_feed_sale_4_slaughter": "slaughter"}, inplace=True)
slaughter_Q1 = slaughter_Q1[["year", "county_fips", "slaughter"]]
print ("max slaughter sale is [{}]".format(slaughter_Q1.slaughter.max()))
slaughter_Q1.head(2)

max slaughter sale is [922743.0]


,year,county_fips,slaughter
0,2017,04005,152.0
1,2017,04015,100.0


In [25]:
human_population = pd.read_pickle(reOrganized_dir + "human_population.sav")
human_population = human_population["human_population"]
human_population.head(2)

,county_fips,year,population
0,01001,1997,41281
1,01001,2002,45909


In [26]:
inventory_RA_herb_NPP_resPop = pd.merge(inventory_RA_herb_NPP, human_population, 
                                        on=["county_fips", "year"], how="left")

inventory_RA_herb_NPP_resPop.head(2)

,year,county_fips,inventory,rangeland_acre,county_area_acre,rangeland_fraction,herb_avg,herb_std,pixel_count,county_total_npp,population
0,2017,01001,8678.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,55448.0
1,2017,01047,14589.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39263.0


In [27]:
USDA_data.keys()

dict_keys(['AgLand', 'wetLand_area', 'feed_expense', 'FarmOperation', 'cattle_inventory', 'source_code', 'Author', 'Date'])

In [28]:
feed_expense = USDA_data["feed_expense"]
feed_expense = feed_expense[["year", "county_fips", "feed_expense"]]
feed_expense.head(2)

,year,county_fips,feed_expense
0,2017,01001,2420000.0
1,2017,01011,4111000.0


In [29]:
inventory_RA_herb_NPP_resPop_feedCost = pd.merge(inventory_RA_herb_NPP_resPop, feed_expense, 
                                                 on=["county_fips", "year"], how="left")

inventory_RA_herb_NPP_resPop_feedCost.head(2)

,year,county_fips,inventory,rangeland_acre,county_area_acre,rangeland_fraction,herb_avg,herb_std,pixel_count,county_total_npp,population,feed_expense
0,2017,01001,8678.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,55448.0,2420000.0
1,2017,01047,14589.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39263.0,11516000.0


In [30]:
slaughter_Q1.head(2)

,year,county_fips,slaughter
0,2017,04005,152.0
1,2017,04015,100.0


In [31]:
inventory_RA_herb_NPP_resPop_feedCost_slaughter = pd.merge(inventory_RA_herb_NPP_resPop_feedCost, 
                                                           slaughter_Q1, 
                                                           on=["county_fips", "year"], how="left")

inventory_RA_herb_NPP_resPop_feedCost_slaughter.head(2)

,year,county_fips,inventory,rangeland_acre,county_area_acre,rangeland_fraction,herb_avg,herb_std,pixel_count,county_total_npp,population,feed_expense,slaughter
0,2017,01001,8678.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,55448.0,2420000.0,NaN
1,2017,01047,14589.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39263.0,11516000.0,NaN


In [32]:
print (inventory_RA_herb_NPP_resPop_feedCost_slaughter.shape)
all_df = inventory_RA_herb_NPP_resPop_feedCost_slaughter.dropna(how="any", inplace=False)
all_df.reset_index(drop=True, inplace=True)
print (all_df.shape)


all_df.drop(["county_area_acre", "herb_std"], axis="columns", inplace=True)
all_df[all_df.pixel_count == 0]

(6464, 13)
(2823, 13)


,year,county_fips,inventory,rangeland_acre,rangeland_fraction,herb_avg,pixel_count,county_total_npp,population,feed_expense,slaughter
1155,2012,51085,3805.0,22.46,0.000074,0.0,0.0,48182.132686,100340.0,5057000.0,154.0
1917,2007,51085,4746.0,22.46,0.000074,0.0,0.0,37086.425829,98830.0,3377000.0,164.0
2796,2002,51085,4520.0,22.46,0.000074,0.0,0.0,40297.205839,91370.0,2768000.0,498.0


### Add Seasonal Weather Variables
Seasonal Variables (<--- just for ```ctrl + F```)

In [33]:
filename = reOrganized_dir + "county_seasonal_temp_ppt_weighted.sav"
seasonal_weather = pd.read_pickle(filename)
print(f"{seasonal_weather.keys() = }")
seasonal_weather = seasonal_weather["seasonal"]
seasonal_weather.head(2)

seasonal_weather.keys() = dict_keys(['seasonal', 'source_code', 'Author', 'Min_file_used', 'Date'])


,county_fips,year,S1_countyMean_total_precip,S2_countyMean_total_precip,S3_countyMean_total_precip,S4_countyMean_total_precip,S1_countyMean_avg_Tavg,S2_countyMean_avg_Tavg,S3_countyMean_avg_Tavg,S4_countyMean_avg_Tavg
0,01003,1979,656.015,678.306,388.477,252.774,11.042778,24.376025,26.330803,15.175087
1,01005,1979,494.41,470.324,277.067,203.357,9.200633,22.690754,24.926689,13.078413


In [34]:
SW_vars = ["S1_countyMean_total_precip",
           "S2_countyMean_total_precip",
           "S3_countyMean_total_precip",
           "S4_countyMean_total_precip",
           "S1_countyMean_avg_Tavg",
           "S2_countyMean_avg_Tavg",
           "S3_countyMean_avg_Tavg",
           "S4_countyMean_avg_Tavg"
          ]

for a_col in SW_vars:
    seasonal_weather[a_col] = seasonal_weather[a_col].astype(float)

In [35]:
all_df = pd.merge(all_df, seasonal_weather, on=["county_fips", "year"], how="left")
all_df.head(2)

,year,county_fips,inventory,rangeland_acre,rangeland_fraction,herb_avg,pixel_count,county_total_npp,population,feed_expense,slaughter,S1_countyMean_total_precip,S2_countyMean_total_precip,S3_countyMean_total_precip,S4_countyMean_total_precip,S1_countyMean_avg_Tavg,S2_countyMean_avg_Tavg,S3_countyMean_avg_Tavg,S4_countyMean_avg_Tavg
0,2017,06053,21257.0,1102652.61,0.51,56.902494,104342.0,2.960837e+09,433803.0,9073000.0,463.0,477.168,39.712,1.493,29.085,10.346378,18.443164,21.800148,13.453609
1,2017,06079,22626.0,1480867.59,0.70,68.116157,117918.0,3.515344e+09,282473.0,8918000.0,185.0,465.353,33.842,7.063,11.931,10.597589,19.325279,22.400623,14.016978


In [36]:
all_df.describe().round(1)

,inventory,rangeland_acre,rangeland_fraction,herb_avg,pixel_count,county_total_npp,population,feed_expense,slaughter,S1_countyMean_total_precip,S2_countyMean_total_precip,S3_countyMean_total_precip,S4_countyMean_total_precip,S1_countyMean_avg_Tavg,S2_countyMean_avg_Tavg,S3_countyMean_avg_Tavg,S4_countyMean_avg_Tavg
count,2823.0,2823.0,2823.0,2823.0,2823.0,2.823000e+03,2823.0,2.823000e+03,2823.0,2823.0,2823.0,2823.0,2823.0,2823.0,2823.0,2823.0,2823.0
mean,18496.1,296702.9,0.2,61.5,20739.0,3.150663e+08,69598.0,2.386068e+07,17544.0,151.1,320.7,137.2,159.6,5.4,19.5,21.6,9.4
std,15109.4,653716.8,0.3,23.9,43269.9,5.031727e+08,288727.8,6.455245e+07,68975.7,122.1,166.7,84.9,127.3,7.3,3.2,3.2,7.9
min,30.0,0.7,0.0,0.0,0.0,7.609000e+02,421.0,3.900000e+04,3.0,6.9,5.3,0.0,0.6,-10.7,6.2,8.9,-4.4
25%,7846.0,2005.9,0.0,41.4,1158.0,4.170482e+06,8301.5,3.641500e+06,327.5,66.0,197.6,77.3,75.0,-0.5,17.5,19.7,3.0
50%,15068.0,51789.9,0.1,61.5,4924.0,9.800547e+07,17437.0,7.946000e+06,1199.0,126.6,300.0,127.2,134.6,4.2,20.1,21.8,7.1
75%,24682.0,310340.8,0.4,84.4,21331.0,4.250045e+08,40226.5,1.841200e+07,6463.0,197.4,429.3,185.5,203.8,12.8,21.8,23.6,16.3
max,161744.0,9325676.4,1.0,96.6,609857.0,4.454698e+09,9700359.0,1.183990e+09,922743.0,1311.4,886.2,619.7,1253.2,20.0,28.8,28.9,22.7


In [37]:
sorted(all_df.columns)

['S1_countyMean_avg_Tavg',
 'S1_countyMean_total_precip',
 'S2_countyMean_avg_Tavg',
 'S2_countyMean_total_precip',
 'S3_countyMean_avg_Tavg',
 'S3_countyMean_total_precip',
 'S4_countyMean_avg_Tavg',
 'S4_countyMean_total_precip',
 'county_fips',
 'county_total_npp',
 'feed_expense',
 'herb_avg',
 'inventory',
 'pixel_count',
 'population',
 'rangeland_acre',
 'rangeland_fraction',
 'slaughter',
 'year']

In [38]:
controls_noHerb = ["population", "feed_expense", "slaughter", "rangeland_acre"]
controls_wHerb =  ["population", "feed_expense", "slaughter", "rangeland_acre", "herb_avg"]

NPP_control_vars_noHerb= ["county_total_npp"] + controls_noHerb
NPP_control_vars_wHerb = ["county_total_npp"] + controls_wHerb

SW_control_vars_noHerb= SW_vars + controls_noHerb
SW_control_vars_wHerb = SW_vars + controls_wHerb

y_var = "inventory"

In [39]:
X = all_df[NPP_control_vars_noHerb]
X = sm.add_constant(X)
Y = all_df[y_var].astype(float)
ks = sm.OLS(Y, X)
ks_result =ks.fit()
ks_result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              inventory   R-squared:                       0.293
Model:                            OLS   Adj. R-squared:                  0.292
Method:                 Least Squares   F-statistic:                     233.9
Date:                Mon, 01 Jan 2024   Prob (F-statistic):          2.39e-209
Time:                        14:00:51   Log-Likelihood:                -30681.
No. Observations:                2823   AIC:                         6.137e+04
Df Residuals:                    2817   BIC:                         6.141e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const             1.361e+04    299.618     45.428      0.000     1.3e+04    1.42e+04
county_total_npp  1.884e-05   9.26e-07     20.356      0.000     1.7e-05    2.07e-05
population          -0.0069      0.001     -8.160      0.000      -0.009      -0.005
feed_expense      1.478e-05   4.36e-06      3.390      0.001    6.23e-06    2.33e-05
slaughter           -0.0095      0.004     -2.336      0.020      -0.017      -0.002
rangeland_acre      -0.0026      0.001     -3.586      0.000      -0.004      -0.001
==============================================================================
Omnibus:                      854.908   Durbin-Watson:                   1.048
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3391.430
Skew:                           1.440   Prob(JB):                         0.00
Kurtosis:                       7.532   Cond. No.                     7.44e+08
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 7.44e+08. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [40]:
del(X, Y, ks, ks_result)

## (unbiased) Normalize so ranges are comparable

In [41]:
all_indp_vars = list(set(NPP_control_vars_noHerb + 
                         NPP_control_vars_wHerb + 
                         SW_control_vars_noHerb + 
                         SW_control_vars_wHerb))
all_indp_vars = sorted(all_indp_vars)
all_indp_vars

['S1_countyMean_avg_Tavg',
 'S1_countyMean_total_precip',
 'S2_countyMean_avg_Tavg',
 'S2_countyMean_total_precip',
 'S3_countyMean_avg_Tavg',
 'S3_countyMean_total_precip',
 'S4_countyMean_avg_Tavg',
 'S4_countyMean_total_precip',
 'county_total_npp',
 'feed_expense',
 'herb_avg',
 'population',
 'rangeland_acre',
 'slaughter']

In [42]:
# standard_indp = preprocessing.scale(all_df[explain_vars_herb]) # this is biased
normal_df = (all_df[all_indp_vars] - all_df[all_indp_vars].mean()) / \
                         all_df[all_indp_vars].std(ddof=1)
normal_df.head(2)

,S1_countyMean_avg_Tavg,S1_countyMean_total_precip,S2_countyMean_avg_Tavg,S2_countyMean_total_precip,S3_countyMean_avg_Tavg,S3_countyMean_total_precip,S4_countyMean_avg_Tavg,S4_countyMean_total_precip,county_total_npp,feed_expense,herb_avg,population,rangeland_acre,slaughter
0,0.680372,2.669750,-0.326817,-1.684917,0.067537,-1.598475,0.510613,-1.024944,5.258176,-0.229080,-0.194525,1.261413,1.232873,-0.247638
1,0.714735,2.573021,-0.049476,-1.720120,0.253895,-1.532847,0.582022,-1.159697,6.360197,-0.231481,0.275308,0.737286,1.811434,-0.251669


In [43]:
normal_cols = [i + j for i, j in zip(all_indp_vars, ["_normal"] * len(all_indp_vars))]
normal_cols

['S1_countyMean_avg_Tavg_normal',
 'S1_countyMean_total_precip_normal',
 'S2_countyMean_avg_Tavg_normal',
 'S2_countyMean_total_precip_normal',
 'S3_countyMean_avg_Tavg_normal',
 'S3_countyMean_total_precip_normal',
 'S4_countyMean_avg_Tavg_normal',
 'S4_countyMean_total_precip_normal',
 'county_total_npp_normal',
 'feed_expense_normal',
 'herb_avg_normal',
 'population_normal',
 'rangeland_acre_normal',
 'slaughter_normal']

In [44]:
all_df[normal_cols] = normal_df
all_df.head(2)

,year,county_fips,inventory,rangeland_acre,rangeland_fraction,herb_avg,pixel_count,county_total_npp,population,feed_expense,...,S3_countyMean_avg_Tavg_normal,S3_countyMean_total_precip_normal,S4_countyMean_avg_Tavg_normal,S4_countyMean_total_precip_normal,county_total_npp_normal,feed_expense_normal,herb_avg_normal,population_normal,rangeland_acre_normal,slaughter_normal
0,2017,06053,21257.0,1102652.61,0.51,56.902494,104342.0,2.960837e+09,433803.0,9073000.0,...,0.067537,-1.598475,0.510613,-1.024944,5.258176,-0.229080,-0.194525,1.261413,1.232873,-0.247638
1,2017,06079,22626.0,1480867.59,0.70,68.116157,117918.0,3.515344e+09,282473.0,8918000.0,...,0.253895,-1.532847,0.582022,-1.159697,6.360197,-0.231481,0.275308,0.737286,1.811434,-0.251669


In [45]:
NPP_control_vars_noHerb_normal = [i + j for i, j in 
                                  zip(NPP_control_vars_noHerb, ["_normal"] * len(NPP_control_vars_noHerb))]

NPP_control_vars_wHerb_normal = [i + j for i, j in 
                                  zip(NPP_control_vars_wHerb, ["_normal"] * len(NPP_control_vars_wHerb))]

SW_control_vars_noHerb_normal = [i + j for i, j in 
                                  zip(SW_control_vars_noHerb, ["_normal"] * len(SW_control_vars_noHerb))]

SW_control_vars_wHerb_normal = [i + j for i, j in 
                                  zip(SW_control_vars_wHerb, ["_normal"] * len(SW_control_vars_wHerb))]

In [46]:
NPP_control_vars_noHerb_normal

['county_total_npp_normal',
 'population_normal',
 'feed_expense_normal',
 'slaughter_normal',
 'rangeland_acre_normal']

In [47]:
normal_cols

['S1_countyMean_avg_Tavg_normal',
 'S1_countyMean_total_precip_normal',
 'S2_countyMean_avg_Tavg_normal',
 'S2_countyMean_total_precip_normal',
 'S3_countyMean_avg_Tavg_normal',
 'S3_countyMean_total_precip_normal',
 'S4_countyMean_avg_Tavg_normal',
 'S4_countyMean_total_precip_normal',
 'county_total_npp_normal',
 'feed_expense_normal',
 'herb_avg_normal',
 'population_normal',
 'rangeland_acre_normal',
 'slaughter_normal']

In [48]:
all_df.head(2)

,year,county_fips,inventory,rangeland_acre,rangeland_fraction,herb_avg,pixel_count,county_total_npp,population,feed_expense,...,S3_countyMean_avg_Tavg_normal,S3_countyMean_total_precip_normal,S4_countyMean_avg_Tavg_normal,S4_countyMean_total_precip_normal,county_total_npp_normal,feed_expense_normal,herb_avg_normal,population_normal,rangeland_acre_normal,slaughter_normal
0,2017,06053,21257.0,1102652.61,0.51,56.902494,104342.0,2.960837e+09,433803.0,9073000.0,...,0.067537,-1.598475,0.510613,-1.024944,5.258176,-0.229080,-0.194525,1.261413,1.232873,-0.247638
1,2017,06079,22626.0,1480867.59,0.70,68.116157,117918.0,3.515344e+09,282473.0,8918000.0,...,0.253895,-1.532847,0.582022,-1.159697,6.360197,-0.231481,0.275308,0.737286,1.811434,-0.251669


In [49]:
NPP_control_vars_noHerb_normal

['county_total_npp_normal',
 'population_normal',
 'feed_expense_normal',
 'slaughter_normal',
 'rangeland_acre_normal']

### NPP and control variables model with herb (normalized)

In [50]:
X_normal = all_df[NPP_control_vars_wHerb_normal]
X_normal = sm.add_constant(X_normal)
Y = all_df[y_var].astype(float)
ks_normal = sm.OLS(Y, X_normal)
ks_normal_result =ks_normal.fit()
ks_normal_result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              inventory   R-squared:                       0.307
Model:                            OLS   Adj. R-squared:                  0.306
Method:                 Least Squares   F-statistic:                     208.0
Date:                Mon, 01 Jan 2024   Prob (F-statistic):          4.32e-220
Time:                        14:01:03   Log-Likelihood:                -30653.
No. Observations:                2823   AIC:                         6.132e+04
Df Residuals:                    2816   BIC:                         6.136e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
===========================================================================================
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
const                     1.85e+04    236.968     78.053      0.000     1.8e+04     1.9e+04
county_total_npp_normal  8495.7632    479.792     17.707      0.000    7554.984    9436.542
population_normal       -1750.4531    242.286     -7.225      0.000   -2225.530   -1275.376
feed_expense_normal       919.2417    278.840      3.297      0.001     372.491    1465.992
slaughter_normal        -1043.0089    282.221     -3.696      0.000   -1596.390    -489.628
rangeland_acre_normal   -1286.8063    465.773     -2.763      0.006   -2200.096    -373.516
herb_avg_normal          1950.4377    261.207      7.467      0.000    1438.261    2462.615
==============================================================================
Omnibus:                      922.698   Durbin-Watson:                   1.081
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3761.652
Skew:                           1.554   Prob(JB):                         0.00
Kurtosis:                       7.724   Cond. No.                         3.94
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### NPP and control variables model No herb (normalized)

In [51]:
X_normal = all_df[NPP_control_vars_noHerb_normal]
X_normal = sm.add_constant(X_normal)
Y = all_df[y_var].astype(float)
ks_normal = sm.OLS(Y, X_normal)
ks_normal_result =ks_normal.fit()
ks_normal_result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              inventory   R-squared:                       0.293
Model:                            OLS   Adj. R-squared:                  0.292
Method:                 Least Squares   F-statistic:                     233.9
Date:                Mon, 01 Jan 2024   Prob (F-statistic):          2.39e-209
Time:                        14:01:12   Log-Likelihood:                -30681.
No. Observations:                2823   AIC:                         6.137e+04
Df Residuals:                    2817   BIC:                         6.141e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
===========================================================================================
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
const                     1.85e+04    239.260     77.305      0.000     1.8e+04     1.9e+04
county_total_npp_normal  9480.9452    465.756     20.356      0.000    8567.688    1.04e+04
population_normal       -1980.0803    242.651     -8.160      0.000   -2455.872   -1504.288
feed_expense_normal       954.2130    281.497      3.390      0.001     402.252    1506.174
slaughter_normal         -654.1531    280.058     -2.336      0.020   -1203.292    -105.014
rangeland_acre_normal   -1675.7025    467.328     -3.586      0.000   -2592.043    -759.362
==============================================================================
Omnibus:                      854.908   Durbin-Watson:                   1.048
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3391.430
Skew:                           1.440   Prob(JB):                         0.00
Kurtosis:                       7.532   Cond. No.                         3.68
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

# Side-by-sides

  - ```ln(y) = f(NPP)```
  - ```ln(y) = f(SW)```
  - ```ln(y) = f(NPP, controls-noHerb)```
  - ```ln(y) = f(SW, controls-noHerb)```

### NPP vs ln(y)

In [55]:
X = all_df["county_total_npp_normal"]
X = sm.add_constant(X)
Y = np.log(all_df[y_var].astype(float))
ks = sm.OLS(Y, X)
ks_result = ks.fit()
ks_result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              inventory   R-squared:                       0.171
Model:                            OLS   Adj. R-squared:                  0.171
Method:                 Least Squares   F-statistic:                     582.7
Date:                Mon, 01 Jan 2024   Prob (F-statistic):          3.45e-117
Time:                        14:01:57   Log-Likelihood:                -3621.9
No. Observations:                2823   AIC:                             7248.
Df Residuals:                    2821   BIC:                             7260.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
===========================================================================================
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
const                       9.4627      0.016    575.770      0.000       9.431       9.495
county_total_npp_normal     0.3968      0.016     24.139      0.000       0.365       0.429
==============================================================================
Omnibus:                      470.000   Durbin-Watson:                   0.859
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              940.317
Skew:                          -1.000   Prob(JB):                    6.50e-205
Kurtosis:                       4.999   Cond. No.                         1.00
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [56]:
del(X, ks, ks_result)

### SW vs ln(y)

In [57]:
controls_noHerb_normal_vars = [i + j for i, j in zip(controls_noHerb, ["_normal"] * len(controls_noHerb))]
controls_wHerb_normal_vars = [i + j for i, j in zip(controls_wHerb, ["_normal"] * len(controls_wHerb))]

SW_vars_normal = [x for x in list(SW_control_vars_noHerb_normal) if not(x in list(controls_noHerb_normal_vars))]
SW_vars_normal

['S1_countyMean_total_precip_normal',
 'S2_countyMean_total_precip_normal',
 'S3_countyMean_total_precip_normal',
 'S4_countyMean_total_precip_normal',
 'S1_countyMean_avg_Tavg_normal',
 'S2_countyMean_avg_Tavg_normal',
 'S3_countyMean_avg_Tavg_normal',
 'S4_countyMean_avg_Tavg_normal']

In [58]:
X = all_df[SW_vars_normal]
X = sm.add_constant(X)
Y = np.log(all_df[y_var].astype(float))
ks = sm.OLS(Y, X)
ks_result = ks.fit()
ks_result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              inventory   R-squared:                       0.088
Model:                            OLS   Adj. R-squared:                  0.086
Method:                 Least Squares   F-statistic:                     34.04
Date:                Mon, 01 Jan 2024   Prob (F-statistic):           1.17e-51
Time:                        14:02:04   Log-Likelihood:                -3756.6
No. Observations:                2823   AIC:                             7531.
Df Residuals:                    2814   BIC:                             7585.
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
=====================================================================================================
                                        coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
const                                 9.4627      0.017    548.271      0.000       9.429       9.497
S1_countyMean_total_precip_normal    -0.0263      0.028     -0.924      0.356      -0.082       0.029
S2_countyMean_total_precip_normal     0.0312      0.023      1.334      0.182      -0.015       0.077
S3_countyMean_total_precip_normal    -0.1000      0.020     -5.081      0.000      -0.139      -0.061
S4_countyMean_total_precip_normal    -0.2647      0.025    -10.744      0.000      -0.313      -0.216
S1_countyMean_avg_Tavg_normal         0.4219      0.092      4.572      0.000       0.241       0.603
S2_countyMean_avg_Tavg_normal        -0.0273      0.072     -0.381      0.703      -0.168       0.113
S3_countyMean_avg_Tavg_normal         0.1379      0.052      2.658      0.008       0.036       0.240
S4_countyMean_avg_Tavg_normal        -0.4178      0.085     -4.942      0.000      -0.584      -0.252
==============================================================================
Omnibus:                      330.910   Durbin-Watson:                   0.941
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              551.974
Skew:                          -0.803   Prob(JB):                    1.38e-120
Kurtosis:                       4.453   Cond. No.                         13.4
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [59]:
del(X, ks, ks_result)

### NPP and controls (no herb) vs ln(y)

In [62]:
NPP_control_vars_noHerb_normal

['county_total_npp_normal',
 'population_normal',
 'feed_expense_normal',
 'slaughter_normal',
 'rangeland_acre_normal']

In [63]:
X = all_df[NPP_control_vars_noHerb_normal]
X = sm.add_constant(X)
Y = np.log(all_df[y_var].astype(float))
ks = sm.OLS(Y, X)
ks_result = ks.fit()
ks_result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              inventory   R-squared:                       0.207
Model:                            OLS   Adj. R-squared:                  0.206
Method:                 Least Squares   F-statistic:                     147.5
Date:                Mon, 01 Jan 2024   Prob (F-statistic):          2.28e-139
Time:                        14:02:36   Log-Likelihood:                -3558.8
No. Observations:                2823   AIC:                             7130.
Df Residuals:                    2817   BIC:                             7165.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
===========================================================================================
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
const                       9.4627      0.016    588.375      0.000       9.431       9.494
county_total_npp_normal     0.5537      0.031     17.685      0.000       0.492       0.615
population_normal          -0.1554      0.016     -9.525      0.000      -0.187      -0.123
feed_expense_normal         0.0612      0.019      3.233      0.001       0.024       0.098
slaughter_normal           -0.0279      0.019     -1.485      0.138      -0.065       0.009
rangeland_acre_normal      -0.1664      0.031     -5.298      0.000      -0.228      -0.105
==============================================================================
Omnibus:                      386.384   Durbin-Watson:                   0.866
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              743.386
Skew:                          -0.856   Prob(JB):                    3.76e-162
Kurtosis:                       4.841   Cond. No.                         3.68
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [64]:
del(X, ks, ks_result)

### SW and controls (no herb) vs ln(y)

In [66]:
SW_control_vars_noHerb_normal

['S1_countyMean_total_precip_normal',
 'S2_countyMean_total_precip_normal',
 'S3_countyMean_total_precip_normal',
 'S4_countyMean_total_precip_normal',
 'S1_countyMean_avg_Tavg_normal',
 'S2_countyMean_avg_Tavg_normal',
 'S3_countyMean_avg_Tavg_normal',
 'S4_countyMean_avg_Tavg_normal',
 'population_normal',
 'feed_expense_normal',
 'slaughter_normal',
 'rangeland_acre_normal']

In [67]:
X = all_df[SW_control_vars_noHerb_normal]
X = sm.add_constant(X)
Y = np.log(all_df[y_var].astype(float))
ks = sm.OLS(Y, X)
ks_result = ks.fit()
ks_result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              inventory   R-squared:                       0.183
Model:                            OLS   Adj. R-squared:                  0.179
Method:                 Least Squares   F-statistic:                     52.43
Date:                Mon, 01 Jan 2024   Prob (F-statistic):          4.97e-114
Time:                        14:02:49   Log-Likelihood:                -3601.8
No. Observations:                2823   AIC:                             7230.
Df Residuals:                    2810   BIC:                             7307.
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
=====================================================================================================
                                        coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
const                                 9.4627      0.016    578.764      0.000       9.431       9.495
S1_countyMean_total_precip_normal     0.0071      0.027      0.262      0.793      -0.046       0.060
S2_countyMean_total_precip_normal     0.1092      0.023      4.711      0.000       0.064       0.155
S3_countyMean_total_precip_normal    -0.0434      0.019     -2.283      0.022      -0.081      -0.006
S4_countyMean_total_precip_normal    -0.2097      0.024     -8.845      0.000      -0.256      -0.163
S1_countyMean_avg_Tavg_normal         0.2927      0.089      3.281      0.001       0.118       0.468
S2_countyMean_avg_Tavg_normal         0.0004      0.068      0.006      0.995      -0.134       0.134
S3_countyMean_avg_Tavg_normal         0.1222      0.050      2.462      0.014       0.025       0.220
S4_countyMean_avg_Tavg_normal        -0.2855      0.081     -3.504      0.000      -0.445      -0.126
population_normal                    -0.1587      0.017     -9.383      0.000      -0.192      -0.126
feed_expense_normal                   0.0521      0.020      2.662      0.008       0.014       0.091
slaughter_normal                     -0.0357      0.020     -1.830      0.067      -0.074       0.003
rangeland_acre_normal                 0.3074      0.019     16.104      0.000       0.270       0.345
==============================================================================
Omnibus:                      522.730   Durbin-Watson:                   0.961
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1312.890
Skew:                          -1.010   Prob(JB):                    8.12e-286
Kurtosis:                       5.661   Cond. No.                         13.9
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [68]:
del(X, ks, ks_result)

### NPP and controls (with herb) vs ln(y)

In [69]:
NPP_control_vars_wHerb_normal

['county_total_npp_normal',
 'population_normal',
 'feed_expense_normal',
 'slaughter_normal',
 'rangeland_acre_normal',
 'herb_avg_normal']

In [70]:
X = all_df[NPP_control_vars_wHerb_normal]
X = sm.add_constant(X)
Y = np.log(all_df[y_var].astype(float))
ks = sm.OLS(Y, X)
ks_result = ks.fit()
ks_result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              inventory   R-squared:                       0.261
Model:                            OLS   Adj. R-squared:                  0.259
Method:                 Least Squares   F-statistic:                     165.6
Date:                Mon, 01 Jan 2024   Prob (F-statistic):          1.02e-180
Time:                        14:03:05   Log-Likelihood:                -3460.4
No. Observations:                2823   AIC:                             6935.
Df Residuals:                    2816   BIC:                             6976.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
===========================================================================================
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
const                       9.4627      0.016    609.150      0.000       9.432       9.493
county_total_npp_normal     0.4303      0.031     13.682      0.000       0.369       0.492
population_normal          -0.1266      0.016     -7.971      0.000      -0.158      -0.095
feed_expense_normal         0.0568      0.018      3.107      0.002       0.021       0.093
slaughter_normal           -0.0766      0.019     -4.143      0.000      -0.113      -0.040
rangeland_acre_normal      -0.1177      0.031     -3.856      0.000      -0.178      -0.058
herb_avg_normal             0.2442      0.017     14.263      0.000       0.211       0.278
==============================================================================
Omnibus:                      279.770   Durbin-Watson:                   0.933
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              543.321
Skew:                          -0.649   Prob(JB):                    1.05e-118
Kurtosis:                       4.713   Cond. No.                         3.94
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [71]:
del(X, ks, ks_result)

### SW and controls (with herb) vs ln(y)

In [72]:
SW_control_vars_wHerb_normal

['S1_countyMean_total_precip_normal',
 'S2_countyMean_total_precip_normal',
 'S3_countyMean_total_precip_normal',
 'S4_countyMean_total_precip_normal',
 'S1_countyMean_avg_Tavg_normal',
 'S2_countyMean_avg_Tavg_normal',
 'S3_countyMean_avg_Tavg_normal',
 'S4_countyMean_avg_Tavg_normal',
 'population_normal',
 'feed_expense_normal',
 'slaughter_normal',
 'rangeland_acre_normal',
 'herb_avg_normal']

In [73]:
X = all_df[SW_control_vars_wHerb_normal]
X = sm.add_constant(X)
Y = np.log(all_df[y_var].astype(float))
ks = sm.OLS(Y, X)
ks_result = ks.fit()
ks_result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              inventory   R-squared:                       0.294
Model:                            OLS   Adj. R-squared:                  0.291
Method:                 Least Squares   F-statistic:                     90.06
Date:                Mon, 01 Jan 2024   Prob (F-statistic):          2.65e-201
Time:                        14:03:26   Log-Likelihood:                -3395.2
No. Observations:                2823   AIC:                             6818.
Df Residuals:                    2809   BIC:                             6902.
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
=====================================================================================================
                                        coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
const                                 9.4627      0.015    622.595      0.000       9.433       9.493
S1_countyMean_total_precip_normal     0.0596      0.025      2.351      0.019       0.010       0.109
S2_countyMean_total_precip_normal     0.1762      0.022      8.088      0.000       0.134       0.219
S3_countyMean_total_precip_normal    -0.0037      0.018     -0.208      0.835      -0.039       0.031
S4_countyMean_total_precip_normal    -0.0789      0.023     -3.444      0.001      -0.124      -0.034
S1_countyMean_avg_Tavg_normal         0.5913      0.084      7.029      0.000       0.426       0.756
S2_countyMean_avg_Tavg_normal        -0.1200      0.064     -1.881      0.060      -0.245       0.005
S3_countyMean_avg_Tavg_normal         0.1564      0.046      3.388      0.001       0.066       0.247
S4_countyMean_avg_Tavg_normal        -0.4479      0.076     -5.883      0.000      -0.597      -0.299
population_normal                    -0.1397      0.016     -8.870      0.000      -0.171      -0.109
feed_expense_normal                   0.0422      0.018      2.317      0.021       0.006       0.078
slaughter_normal                     -0.0809      0.018     -4.434      0.000      -0.117      -0.045
rangeland_acre_normal                 0.2992      0.018     16.860      0.000       0.264       0.334
herb_avg_normal                       0.4147      0.020     21.041      0.000       0.376       0.453
==============================================================================
Omnibus:                      424.903   Durbin-Watson:                   1.120
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              947.138
Skew:                          -0.875   Prob(JB):                    2.15e-206
Kurtosis:                       5.234   Cond. No.                         14.5
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## Include lag ```NPP``` in the model

Do we have annual county-level NPP?